<p style="text-align: left; font-size: 32px; font-weight: bold;">
    Creating animated gifs of vertical reflectivity columns on advanced complex plots
</p>

# Overview
### Within this notebook, we will cover:
#### 1. How to create a complex plot with vertical reflectivity columns for specified points, reflectivity PPI, and derived rainfall rate.

# Prerequisites 


| Concepts | Importance | Notes |
| -------- | ---------- | ----- |
| [Quickstart: Zero to Python](https://foundations.projectpythia.org/foundations/quickstart.html) | Required | For loops, lists |
| [Matplotlib Basics](https://link-to-matplotlib-basics) | Required | Basic plotting |
| [Py-ART Basics](https://link-to-pyart-basics) | Required | IO/Visualization |
| [The Basics of Weather Radar](https://projectpythia.org/radar-cookbook/notebooks/radar-basics/radar-basics.html) | Required | Competency with Radar and its products |
| [Extract a radar column above a point](https://arm-doe.github.io/pyart/examples/retrieve/plot_column_subset.html) | Required | Understanding of vertical columns above a radar or point |
* **Time to learn:** 45 Minutes


# *NOTE! This is a continuation of the animated GIF saga. Remember to familiarize yourself with each of the prior gif making notebooks to better understand how this one works.

# Imports

In [ ]:
import os
import numpy as np
import pyart
import fsspec
import xarray as xr
from datetime import datetime, timedelta
import warnings
from io import BytesIO
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import cartopy.crs as ccrs
from metpy.plots import USCOUNTIES
from PIL import Image
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

## The initial setup for this product will be similar to the other notebooks, with some notable differences

Instead of directly pulling all of the data from the AWS and using it directly, we will be saving the data to temporary netCDF files for ease of access. This process will be explained throughout the demonstration

To initialize this process, we will use the same code as before to pull the radar files and read them. Although this time we are filtering out the mdm files due to errors in plotting. 

In [ ]:
fs = fsspec.filesystem("s3", anon=True)

# Define the start and end dates and hours
start_date = datetime(2024, 6, 5, 4, 0)  # YYYY/MM/DD HH
end_date = datetime(2024, 6, 5, 10, 0)   # YYYY/MM/DD HH
station = 'KLOT'
latitude = [41.700937896518866, 41.704120]
longitude = [-87.99578103231573, -87.968328]
labels = ['Tower', 'SMC']
markers = ['^', 'o']
colors = ['magenta', 'cyan']

# Generate the list of files for the specified date and hour range
files = []
current_date = start_date

while current_date <= end_date:
    date_str = current_date.strftime("%Y/%m/%d")
    date_str_compact = current_date.strftime("%Y%m%d")

    if current_date.date() == start_date.date():
        start_hour = start_date.hour
    else:
        start_hour = 0

    if current_date.date() == end_date.date():
        end_hour = end_date.hour
    else:
        end_hour = 23

    for hour in range(start_hour, end_hour + 1):
        hour_str = f"{hour:02d}"
        all_files = fs.glob(f"s3://noaa-nexrad-level2/{date_str}/{station}/{station}{date_str_compact}_{hour_str}*")
        filtered_files = [f for f in all_files if not f.endswith('_MDM')]
        files += sorted(filtered_files)

    current_date = current_date.replace(hour=0) + timedelta(days=1)

# Print the selected files
print("Selected files:")
for file in files:
    print(file)

# Function to check and print the fields in the first radar file
def check_radar_fields(file_path):
    try:
        with fs.open(file_path, 'rb') as f:
            radar_data = f.read()
        radar_file = BytesIO(radar_data)
        radar = pyart.io.read_nexrad_archive(radar_file)
        print(f"Fields in radar data from {file_path}:")
        print(list(radar.fields.keys()))
    except Exception as e:
        print(f"Failed to read radar data from {file_path}: {e}")

# Check the fields for the first file
if files:
    check_radar_fields(files[0])

# Function to read radar data
def read_radar_data(file_path):
    try:
        with fs.open(file_path, 'rb') as f:
            radar_data = f.read()
        radar_file = BytesIO(radar_data)
        radar = pyart.io.read_nexrad_archive(radar_file)
        print(f"Successfully read radar data from {file_path}")
        return radar
    except Exception as e:
        print(f"Failed to read radar data from {file_path}: {e}")
        return None

# Collect data for the NetCDF files
time_height_data_tower = []
time_height_data_smc = []
times = []
heights = None  # Initialize heights to None

## netCDF file creation

#### To create the netCDF file, we will extract the radar data from the files and save the reflectivity data for each points, preserving the spatial and temporal data by indexing the variable by time and height. This is essential for plotting our vertical column data

NAN values are also filtered out of the radar to prevent gaps in the data.

Two seperate netCDF files will be created, one for each point. If you would like to create more or less depending on your personal needs, this code can be modified accordingly. 
Different product data can be saved onto these netCDF files for whatever use case.

In [ ]:
for file in files:
    radar = read_radar_data(file)
    if radar is None:
        continue

    # Append the corresponding time once per file
    radar_time = pd.to_datetime(radar.time['units'].split('since ')[-1]) + pd.to_timedelta(radar.time['data'][0], unit='s')
    times.append(np.datetime64(radar_time))

    for idx in [0, 1]:
        # Extract vertical profile
        profile = pyart.util.column_vertical_profile(radar, latitude[idx], longitude[idx], azimuth_spread=3, spatial_spread=3)
        reflectivity = profile['reflectivity']

        # Get the heights of the sweeps if not already obtained
        if heights is None:
            heights = radar.gate_altitude['data'][0, :]

        # Append the profile to the respective array
        if idx == 0:
            time_height_data_tower.append(reflectivity)
        else:
            time_height_data_smc.append(reflectivity)

# Determine the maximum profile length
max_length_tower = max(len(profile) for profile in time_height_data_tower)
max_length_smc = max(len(profile) for profile in time_height_data_smc)

# Pad profiles with NaNs to match the maximum length
padded_time_height_data_tower = np.full((len(time_height_data_tower), max_length_tower), np.nan)
padded_time_height_data_smc = np.full((len(time_height_data_smc), max_length_smc), np.nan)

for i, profile in enumerate(time_height_data_tower):
    padded_time_height_data_tower[i, :len(profile)] = profile
for i, profile in enumerate(time_height_data_smc):
    padded_time_height_data_smc[i, :len(profile)] = profile

# Convert collected data to 2D arrays
time_height_data_tower = np.array(padded_time_height_data_tower)
time_height_data_smc = np.array(padded_time_height_data_smc)

# Ensure heights array matches the padded data's second dimension
heights_tower = np.linspace(heights[0], heights[-1], max_length_tower)
heights_smc = np.linspace(heights[0], heights[-1], max_length_smc)

# Create the xarray Datasets
ds_tower = xr.Dataset(
    {
        "reflectivity": (["time", "height"], time_height_data_tower)
    },
    coords={
        "time": times[:len(time_height_data_tower)],  # Ensure time dimension matches the data
        "height": heights_tower
    }
)

ds_smc = xr.Dataset(
    {
        "reflectivity": (["time", "height"], time_height_data_smc)
    },
    coords={
        "time": times[:len(time_height_data_smc)],  # Ensure time dimension matches the data
        "height": heights_smc
    }
)

# Save the datasets to NetCDF files
output_file_tower = "combined_radar_Tower.nc"
output_file_smc = "combined_radar_SMC.nc"
ds_tower.to_netcdf(output_file_tower)
ds_smc.to_netcdf(output_file_smc)
print(f"Saved combined dataset to {output_file_tower}")
print(f"Saved combined dataset to {output_file_smc}")

## Initializing the netcDF files for use

Now that we have our netCDF files created and saved, we can use them for data extraction and plotting purposes. 

The data is extracted and converted to numpy arrays for use in plotting. 

In [ ]:
# Load the NetCDF files
ds_tower = xr.open_dataset(output_file_tower)
ds_smc = xr.open_dataset(output_file_smc)

# Extract data from the datasets
times = ds_tower['time'].values
heights_tower = ds_tower['height'].values
reflectivity_tower = ds_tower['reflectivity'].values

heights_smc = ds_smc['height'].values
reflectivity_smc = ds_smc['reflectivity'].values

# Convert to numpy arrays
padded_time_height_data_tower = np.array(reflectivity_tower)
padded_time_height_data_smc = np.array(reflectivity_smc)

reflectivity_tower = np.nan_to_num(padded_time_height_data_tower)
reflectivity_smc = np.nan_to_num(padded_time_height_data_smc)

## Calculation of Z-R relationship for derived rainfall rate caluclations

The NAN values have to be filtered for this step, otherwise you will see gaps in your rainfall rate line. 

For this example, since we are dealing with summertime convection, we will use the Z-R relationship for deep convection rather than the Marshall-Palmer stratiform formula. Although, feel free to change this if you're analyzing a stratiform event. 

The code succeeding the ZR calculation initializes the meshes and arrays for gradual data addition.
This also includes the intialization of a rainfall accumulation calculation (not 100% accurate as its radar derived)

A directory called frames, with a subdirectory called vertical will be created to store each of the frames created by the code

In [ ]:
Z_tower = 10 ** (reflectivity_tower / 10)  # Convert dBZ to Z
R_tower = (Z_tower / 300) ** (1 / 1.4)  # Calculate rainfall rate

Z_smc = 10 ** (reflectivity_smc / 10)  # Convert dBZ to Z
R_smc = (Z_smc / 300) ** (1 / 1.4)  # Calculate rainfall rate

# Initialize total rainfall variables
total_rainfall_tower = 0
total_rainfall_smc = 0

### Plotting
frames_dir = 'frames/Vertical'
os.makedirs(frames_dir, exist_ok=True)
frames = []

# Prepare the meshes for plotting
time_mesh_smc, height_mesh_smc = np.meshgrid(mdates.date2num(times), heights_smc)
time_mesh_tower, height_mesh_tower = np.meshgrid(mdates.date2num(times), heights_tower)

# Initialize empty arrays for the gradual addition of data
incremental_data_smc = np.full_like(padded_time_height_data_smc, np.nan)
incremental_data_tower = np.full_like(padded_time_height_data_tower, np.nan)

## Plotting 
Now we will plot the data onto the figure. This will be a 5 panel plot with one main radar plot, and 4 derived plots. 

Each variable is properly annotated within the code for ease of modification.

#### The time height plots visualize the reflectivity column over each location specified earlier in the code. It will create a figure that shows the change in vertical reflectivity columns with respect to time to visualize the position of the convective precipitaton core (should one exist) over the area.

#### The rainfall rate plots will visualize the change in rainfall rate as calculated by the Z-R relationship. 

Each frame created for the gif will be saved and appended to the frames list created earlier.
An annotation will appear in the top right corner of each of the rainfall rate boxes with a calculated accumulation

In [ ]:
for i in range(len(files)):
    fig = plt.figure(figsize=(20, 10))

    # Plot map with radar data
    radar = read_radar_data(files[i])
    if radar is None:
        continue
    ax1 = fig.add_axes([0.05, 0.1, 0.4, 0.8], projection=ccrs.PlateCarree())
    display = pyart.graph.RadarMapDisplay(radar)
    try:
        display.plot_ppi_map('reflectivity',
                             sweep=0,
                             vmin=10,
                             vmax=65,
                             ax=ax1,
                             title=f'Reflectivity for {station} {times[i]}',
                             cmap='pyart_ChaseSpectral',
                             colorbar_flag=False)  # Disable the built-in colorbar
        mappable = display.plots[0]
    except Exception as e:
        print(f"Error plotting radar data for file {file}: {e}")
        plt.close(fig)
        continue

    ax1.set_xlim(-88.5, -87.8)  # Adjust the xlim to center over Champaign, IL
    ax1.set_ylim(39.8, 40.5)  # Adjust the ylim to center over Champaign, IL
    ax1.add_feature(USCOUNTIES, linewidth=0.5)
    for lat, lon, label, marker, color in zip(latitude, longitude, labels, markers, colors):
        ax1.plot(lon, lat, marker, label=label, color=color, transform=ccrs.PlateCarree(), markersize=10)
    ax1.legend(loc='upper right', fontsize='large', title='Locations')
    cbar = plt.colorbar(mappable, ax=ax1, orientation='horizontal', fraction=0.046, pad=0.04)
    cbar.set_label('Equivalent Reflectivity Factor (dBZ)')

    # Create a time-height profile plot for SMC
    ax2 = fig.add_axes([0.55, 0.7, 0.4, 0.2])
    c_smc = ax2.pcolormesh(time_mesh_smc, height_mesh_smc, incremental_data_smc.T, shading='auto', cmap='pyart_NWSRef', vmin=0, vmax=65)
    fig.colorbar(c_smc, ax=ax2, label='Reflectivity (dBZ)')
    ax2.set_xlim(time_mesh_smc.min(), time_mesh_smc.max())
    ax2.set_ylim(height_mesh_smc.min(), height_mesh_smc.max())
    ax2.set_xlabel('Time')
    ax2.set_ylabel('Height (ft)')
    ax2.xaxis_date()
    ax2.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))
    ax2.set_title('Reflectivity Profile Above SMC Site')
    fig.autofmt_xdate()

    # Create a time-height profile plot for Tower
    ax3 = fig.add_axes([0.55, 0.4, 0.4, 0.2])
    c_tower = ax3.pcolormesh(time_mesh_tower, height_mesh_tower, incremental_data_tower.T, shading='auto', cmap='pyart_NWSRef', vmin=0, vmax=65)
    fig.colorbar(c_tower, ax=ax3, label='Reflectivity (dBZ)')
    ax3.set_xlim(time_mesh_tower.min(), time_mesh_tower.max())
    ax3.set_ylim(height_mesh_tower.min(), height_mesh_tower.max())
    ax3.set_xlabel('Time')
    ax3.set_ylabel('Height (ft)')
    ax3.xaxis_date()
    ax3.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))
    ax3.set_title('Reflectivity Profile Above Tower Site')
    fig.autofmt_xdate()

    # Calculate the time interval in hours
    if i == 0:
        time_interval = 0
    else:
        time_interval = (times[i] - times[i-1]).astype('timedelta64[s]').astype(float) / 3600

    # Create a rainfall rate plot for SMC
    ax4 = fig.add_axes([0.55, 0.1, 0.4, 0.2])
    ax4.plot(times[:i+1], R_smc[:i+1, 0], color='blue')
    ax4.set_xlim(times[0], times[-1])
    ax4.set_ylim(-5, 80)
    ax4.set_yticks(np.arange(0, 81, 10))
    ax4.set_yticklabels([f'{int(y)}' for y in np.arange(0, 81, 10)])
    ax4.set_xlabel('Time')
    ax4.set_ylabel('Rainfall Rate (mm/hr)')
    ax4.xaxis_date()
    ax4.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))
    ax4.set_title('Rainfall Rate at Ground Level (SMC)')
    fig.autofmt_xdate()

    # Update total rainfall for SMC
    if i > 0:
        total_rainfall_smc += R_smc[i, 0] * time_interval

    # Add total rainfall annotation
    ax4.annotate(f'Total: {total_rainfall_smc:.2f} mm', xy=(.993, 0.895), xycoords='axes fraction', fontsize=12,
                 horizontalalignment='right', verticalalignment='bottom', bbox=dict(facecolor='white', alpha=0.6))

    # Create a rainfall rate plot for Tower
    ax5 = fig.add_axes([0.55, -0.2, 0.4, 0.2])
    ax5.plot(times[:i+1], R_tower[:i+1, 0], color='blue')
    ax5.set_xlim(times[0], times[-1])
    ax5.set_ylim(-5, 80)
    ax5.set_yticks(np.arange(0, 81, 10))
    ax5.set_yticklabels([f'{int(y)}' for y in np.arange(0, 81, 10)])
    ax5.set_xlabel('Time')
    ax5.set_ylabel('Rainfall Rate (mm/hr)')
    ax5.xaxis_date()
    ax5.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))
    ax5.set_title('Rainfall Rate at Ground Level (Tower)')
    fig.autofmt_xdate()

    # Update total rainfall for Tower
    if i > 0:
        total_rainfall_tower += R_tower[i, 0] * time_interval

    # Add total rainfall annotation
    ax5.annotate(f'Total: {total_rainfall_tower:.2f} mm', xy=(.993, 0.895), xycoords='axes fraction', fontsize=12,
                 horizontalalignment='right', verticalalignment='bottom', bbox=dict(facecolor='white', alpha=0.6))

    # Update data
    incremental_data_smc[:i+1, :] = padded_time_height_data_smc[:i+1, :]
    incremental_data_tower[:i+1, :] = padded_time_height_data_tower[:i+1, :]

    c_smc.set_array(incremental_data_smc.T.flatten())
    c_tower.set_array(incremental_data_tower.T.flatten())

    plt.tight_layout()
    frame_filename = f'combined_frame_{i}.png'
    plt.savefig(frame_filename, bbox_inches='tight')
    frames.append(frame_filename)
    plt.close(fig)

    print(f"Saved frame {i+1}/{len(files)}: {frame_filename}")

## GIF creation and cleanup

Each frame will be temporarily saved to the current working directory (unless otherwise changed) and will remain until all frames have been created, and the GIF is generated, after which the frames will be deleted, the NETCDF files will be closed and deleted as well. 

### IF you want to keep the netCDF files for whatever purpose, feel free to remove or comment the code to remove them out.

In [ ]:
images = [Image.open(frame) for frame in frames]
gif_filename = 'combined_animation.gif'
images[0].save(gif_filename, save_all=True, append_images=images[1:], duration=375, loop=0)

# Cleanup
for frame in frameas:
    os.remove(frame)

# Close and delete the NetCDF files
ds_tower.close()
ds_smc.close()
os.remove(output_file_tower)
os.remove(output_file_smc)

print(f'Animated GIF saved as {gif_filename}')

# Full Code

In [ ]:
import os
import numpy as np
import pyart
import fsspec
import xarray as xr
from datetime import datetime, timedelta
import warnings
from io import BytesIO
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import cartopy.crs as ccrs
from metpy.plots import USCOUNTIES
from PIL import Image

warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

# Setup the S3 filesystem
fs = fsspec.filesystem("s3", anon=True)

# Define the start and end dates and hours
start_date = datetime(2024, 6, 13, 22, 0)  # YYYY/MM/DD HH
end_date = datetime(2024, 6, 13, 23, 0)   # YYYY/MM/DD HH
station = 'KILX'
latitude = [40.0954, 40.1113739013672]
longitude = [-88.2002, -88.2605056762695]
labels = ['CP-58', 'CP-59']
markers = ['^', 'o']
colors = ['magenta', 'cyan']

# Generate the list of files for the specified date and hour range
files = []
current_date = start_date

while current_date <= end_date:
    date_str = current_date.strftime("%Y/%m/%d")
    date_str_compact = current_date.strftime("%Y%m%d")

    if current_date.date() == start_date.date():
        start_hour = start_date.hour
    else:
        start_hour = 0

    if current_date.date() == end_date.date():
        end_hour = end_date.hour
    else:
        end_hour = 23

    for hour in range(start_hour, end_hour + 1):
        hour_str = f"{hour:02d}"
        all_files = fs.glob(f"s3://noaa-nexrad-level2/{date_str}/{station}/{station}{date_str_compact}_{hour_str}*")
        filtered_files = [f for f in all_files if not f.endswith('_MDM')]
        files += sorted(filtered_files)

    current_date = current_date.replace(hour=0) + timedelta(days=1)

# Print the selected files
print("Selected files:")
for file in files:
    print(file)

# Function to check and print the fields in the first radar file
def check_radar_fields(file_path):
    try:
        with fs.open(file_path, 'rb') as f:
            radar_data = f.read()
        radar_file = BytesIO(radar_data)
        radar = pyart.io.read_nexrad_archive(radar_file)
        print(f"Fields in radar data from {file_path}:")
        print(list(radar.fields.keys()))
    except Exception as e:
        print(f"Failed to read radar data from {file_path}: {e}")

# Check the fields for the first file
if files:
    check_radar_fields(files[0])

# Function to read radar data
def read_radar_data(file_path):
    try:
        with fs.open(file_path, 'rb') as f:
            radar_data = f.read()
        radar_file = BytesIO(radar_data)
        radar = pyart.io.read_nexrad_archive(radar_file)
        print(f"Successfully read radar data from {file_path}")
        return radar
    except Exception as e:
        print(f"Failed to read radar data from {file_path}: {e}")
        return None

# Collect data for the NetCDF files
time_height_data_tower = []
time_height_data_smc = []
times = []
heights = None  # Initialize heights to None

for file in files:
    radar = read_radar_data(file)
    if radar is None:
        continue

    # Append the corresponding time once per file
    radar_time = pd.to_datetime(radar.time['units'].split('since ')[-1]) + pd.to_timedelta(radar.time['data'][0], unit='s')
    times.append(np.datetime64(radar_time))

    for idx in [0, 1]:
        # Extract vertical profile
        profile = pyart.util.column_vertical_profile(radar, latitude[idx], longitude[idx], azimuth_spread=3, spatial_spread=3)
        reflectivity = profile['reflectivity']

        # Get the heights of the sweeps if not already obtained
        if heights is None:
            heights = radar.gate_altitude['data'][0, :]

        # Append the profile to the respective array
        if idx == 0:
            time_height_data_tower.append(reflectivity)
        else:
            time_height_data_smc.append(reflectivity)

# Determine the maximum profile length
max_length_tower = max(len(profile) for profile in time_height_data_tower)
max_length_smc = max(len(profile) for profile in time_height_data_smc)

# Pad profiles with NaNs to match the maximum length
padded_time_height_data_tower = np.full((len(time_height_data_tower), max_length_tower), np.nan)
padded_time_height_data_smc = np.full((len(time_height_data_smc), max_length_smc), np.nan)

for i, profile in enumerate(time_height_data_tower):
    padded_time_height_data_tower[i, :len(profile)] = profile
for i, profile in enumerate(time_height_data_smc):
    padded_time_height_data_smc[i, :len(profile)] = profile

# Convert collected data to 2D arrays
time_height_data_tower = np.array(padded_time_height_data_tower)
time_height_data_smc = np.array(padded_time_height_data_smc)

# Ensure heights array matches the padded data's second dimension
heights_tower = np.linspace(heights[0], heights[-1], max_length_tower)
heights_smc = np.linspace(heights[0], heights[-1], max_length_smc)

# Create the xarray Datasets
ds_tower = xr.Dataset(
    {
        "reflectivity": (["time", "height"], time_height_data_tower)
    },
    coords={
        "time": times[:len(time_height_data_tower)],  # Ensure time dimension matches the data
        "height": heights_tower
    }
)

ds_smc = xr.Dataset(
    {
        "reflectivity": (["time", "height"], time_height_data_smc)
    },
    coords={
        "time": times[:len(time_height_data_smc)],  # Ensure time dimension matches the data
        "height": heights_smc
    }
)

# Save the datasets to NetCDF files
output_file_tower = "combined_radar_Tower.nc"
output_file_smc = "combined_radar_SMC.nc"
ds_tower.to_netcdf(output_file_tower)
ds_smc.to_netcdf(output_file_smc)
print(f"Saved combined dataset to {output_file_tower}")
print(f"Saved combined dataset to {output_file_smc}")

### Start of the second part of the code

# Load the NetCDF files
ds_tower = xr.open_dataset(output_file_tower)
ds_smc = xr.open_dataset(output_file_smc)

# Extract data from the datasets
times = ds_tower['time'].values
heights_tower = ds_tower['height'].values
reflectivity_tower = ds_tower['reflectivity'].values

heights_smc = ds_smc['height'].values
reflectivity_smc = ds_smc['reflectivity'].values

# Convert to numpy arrays
padded_time_height_data_tower = np.array(reflectivity_tower)
padded_time_height_data_smc = np.array(reflectivity_smc)

# Replace NaN values with zeros for Z-R calculation
reflectivity_tower = np.nan_to_num(padded_time_height_data_tower)
reflectivity_smc = np.nan_to_num(padded_time_height_data_smc)

# Calculate rainfall rate using Z-R relationship (for the lowest height level)
Z_tower = 10 ** (reflectivity_tower / 10)  # Convert dBZ to Z
R_tower = (Z_tower / 300) ** (1 / 1.4)  # Calculate rainfall rate

Z_smc = 10 ** (reflectivity_smc / 10)  # Convert dBZ to Z
R_smc = (Z_smc / 300) ** (1 / 1.4)  # Calculate rainfall rate

# Initialize total rainfall variables
total_rainfall_tower = 0
total_rainfall_smc = 0

### Plotting
frames_dir = 'frames/Vertical'
os.makedirs(frames_dir, exist_ok=True)
frames = []

# Prepare the meshes for plotting
time_mesh_smc, height_mesh_smc = np.meshgrid(mdates.date2num(times), heights_smc)
time_mesh_tower, height_mesh_tower = np.meshgrid(mdates.date2num(times), heights_tower)

# Initialize empty arrays for the gradual addition of data
incremental_data_smc = np.full_like(padded_time_height_data_smc, np.nan)
incremental_data_tower = np.full_like(padded_time_height_data_tower, np.nan)

for i in range(len(files)):
    fig = plt.figure(figsize=(20, 10))

    # Plot map with radar data
    radar = read_radar_data(files[i])
    if radar is None:
        continue
    ax1 = fig.add_axes([0.05, 0.1, 0.4, 0.8], projection=ccrs.PlateCarree())
    display = pyart.graph.RadarMapDisplay(radar)
    try:
        display.plot_ppi_map('reflectivity',
                             sweep=0,
                             vmin=10,
                             vmax=65,
                             ax=ax1,
                             title=f'Reflectivity for {station} {times[i]}',
                             cmap='pyart_ChaseSpectral',
                             colorbar_flag=False)  # Disable the built-in colorbar
        mappable = display.plots[0]
    except Exception as e:
        print(f"Error plotting radar data for file {file}: {e}")
        plt.close(fig)
        continue

    ax1.set_xlim(-88.5, -87.8)  # Adjust the xlim to center over Champaign, IL
    ax1.set_ylim(39.8, 40.5)  # Adjust the ylim to center over Champaign, IL
    ax1.add_feature(USCOUNTIES, linewidth=0.5)
    for lat, lon, label, marker, color in zip(latitude, longitude, labels, markers, colors):
        ax1.plot(lon, lat, marker, label=label, color=color, transform=ccrs.PlateCarree(), markersize=10)
    ax1.legend(loc='upper right', fontsize='large', title='Locations')
    cbar = plt.colorbar(mappable, ax=ax1, orientation='horizontal', fraction=0.046, pad=0.04)
    cbar.set_label('Equivalent Reflectivity Factor (dBZ)')

    # Create a time-height profile plot for SMC
    ax2 = fig.add_axes([0.55, 0.7, 0.4, 0.2])
    c_smc = ax2.pcolormesh(time_mesh_smc, height_mesh_smc, incremental_data_smc.T, shading='auto', cmap='pyart_NWSRef', vmin=0, vmax=65)
    fig.colorbar(c_smc, ax=ax2, label='Reflectivity (dBZ)')
    ax2.set_xlim(time_mesh_smc.min(), time_mesh_smc.max())
    ax2.set_ylim(height_mesh_smc.min(), height_mesh_smc.max())
    ax2.set_xlabel('Time')
    ax2.set_ylabel('Height (ft)')
    ax2.xaxis_date()
    ax2.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))
    ax2.set_title('Reflectivity Profile Above SMC Site')
    fig.autofmt_xdate()

    # Create a time-height profile plot for Tower
    ax3 = fig.add_axes([0.55, 0.4, 0.4, 0.2])
    c_tower = ax3.pcolormesh(time_mesh_tower, height_mesh_tower, incremental_data_tower.T, shading='auto', cmap='pyart_NWSRef', vmin=0, vmax=65)
    fig.colorbar(c_tower, ax=ax3, label='Reflectivity (dBZ)')
    ax3.set_xlim(time_mesh_tower.min(), time_mesh_tower.max())
    ax3.set_ylim(height_mesh_tower.min(), height_mesh_tower.max())
    ax3.set_xlabel('Time')
    ax3.set_ylabel('Height (ft)')
    ax3.xaxis_date()
    ax3.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))
    ax3.set_title('Reflectivity Profile Above Tower Site')
    fig.autofmt_xdate()

    # Calculate the time interval in hours
    if i == 0:
        time_interval = 0
    else:
        time_interval = (times[i] - times[i-1]).astype('timedelta64[s]').astype(float) / 3600

    # Create a rainfall rate plot for SMC
    ax4 = fig.add_axes([0.55, 0.1, 0.4, 0.2])
    ax4.plot(times[:i+1], R_smc[:i+1, 0], color='blue')
    ax4.set_xlim(times[0], times[-1])
    ax4.set_ylim(-5, 80)
    ax4.set_yticks(np.arange(0, 81, 10))
    ax4.set_yticklabels([f'{int(y)}' for y in np.arange(0, 81, 10)])
    ax4.set_xlabel('Time')
    ax4.set_ylabel('Rainfall Rate (mm/hr)')
    ax4.xaxis_date()
    ax4.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))
    ax4.set_title('Rainfall Rate at Ground Level (SMC)')
    fig.autofmt_xdate()

    # Update total rainfall for SMC
    if i > 0:
        total_rainfall_smc += R_smc[i, 0] * time_interval

    # Add total rainfall annotation
    ax4.annotate(f'Total: {total_rainfall_smc:.2f} mm', xy=(.993, 0.895), xycoords='axes fraction', fontsize=12,
                 horizontalalignment='right', verticalalignment='bottom', bbox=dict(facecolor='white', alpha=0.6))

    # Create a rainfall rate plot for Tower
    ax5 = fig.add_axes([0.55, -0.2, 0.4, 0.2])
    ax5.plot(times[:i+1], R_tower[:i+1, 0], color='blue')
    ax5.set_xlim(times[0], times[-1])
    ax5.set_ylim(-5, 80)
    ax5.set_yticks(np.arange(0, 81, 10))
    ax5.set_yticklabels([f'{int(y)}' for y in np.arange(0, 81, 10)])
    ax5.set_xlabel('Time')
    ax5.set_ylabel('Rainfall Rate (mm/hr)')
    ax5.xaxis_date()
    ax5.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))
    ax5.set_title('Rainfall Rate at Ground Level (Tower)')
    fig.autofmt_xdate()

    # Update total rainfall for Tower
    if i > 0:
        total_rainfall_tower += R_tower[i, 0] * time_interval

    # Add total rainfall annotation
    ax5.annotate(f'Total: {total_rainfall_tower:.2f} mm', xy=(.993, 0.895), xycoords='axes fraction', fontsize=12,
                 horizontalalignment='right', verticalalignment='bottom', bbox=dict(facecolor='white', alpha=0.6))

    # Update data
    incremental_data_smc[:i+1, :] = padded_time_height_data_smc[:i+1, :]
    incremental_data_tower[:i+1, :] = padded_time_height_data_tower[:i+1, :]

    c_smc.set_array(incremental_data_smc.T.flatten())
    c_tower.set_array(incremental_data_tower.T.flatten())

    plt.tight_layout()
    frame_filename = f'combined_frame_{i}.png'
    plt.savefig(frame_filename, bbox_inches='tight')
    frames.append(frame_filename)
    plt.close(fig)

    print(f"Saved frame {i+1}/{len(files)}: {frame_filename}")

# Create animated GIF
images = [Image.open(frame) for frame in frames]
gif_filename = 'combined_animation.gif'
images[0].save(gif_filename, save_all=True, append_images=images[1:], duration=375, loop=0)

# Cleanup
for frame in frames:
    os.remove(frame)

# Close and delete the NetCDF files
ds_tower.close()
ds_smc.close()
os.remove(output_file_tower)
os.remove(output_file_smc)

print(f'Animated GIF saved as {gif_filename}')


# Summary
Within this example, we combined the previous knowledge on animated gif making with Py-ART and made a complex plot pulling vertical reflectivity columns over sites and calculating and plotting rainfall rates.

## What comes next?

Other tutorials for animated GIFs of products, if desired, can be request at 